# The Best Friend Of An Alchemist

While working on this competition, I've encountered an intersting library called [`tsfresh`](https://github.com/blue-yonder/tsfresh) that helps one to automate feature engeneering process a bit. Though it didn't show too impressive result, it does a pretty decent job in generating features without too many efforts.

In this kernel, a simple approach is applied to generate a bunch of features from the "raw" data and see how they work. 

> **Tip:** Before I discovered how to override the default list of computed metrics, I had wrapped the library's functions with some custom code to compute specific features only, and speed up the process a bit. You probably could achieve the same result using [different settings](https://github.com/blue-yonder/tsfresh/blob/master/tsfresh/feature_extraction/settings.py) instead, or writing your own.

In [1]:
from collections import ChainMap
from multiprocessing import cpu_count
from pathlib import Path

In [2]:
import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn.externals.joblib import Parallel, delayed
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, StratifiedKFold
from tqdm import tqdm_notebook as tqdm
from tsfresh.feature_extraction.feature_calculators import *
from tsfresh.feature_selection.relevance import calculate_relevance_table

In [3]:
seed = 1
np.random.seed(seed)

## Dataset And Features

First of all, we read the files and drop the irrelavant columns.

In [5]:
ROOT = Path.cwd().parent/'input'
SAMPLE = ROOT/'sample_submission.csv'
TRAIN = ROOT/'X_train.csv'
TARGET = ROOT/'y_train.csv'
TEST = ROOT/'X_test.csv'

ID_COLS = ['series_id', 'measurement_number']

x_cols = {
    'series_id': np.uint32,
    'measurement_number': np.uint32,
    'orientation_X': np.float32,
    'orientation_Y': np.float32,
    'orientation_Z': np.float32,
    'orientation_W': np.float32,
    'angular_velocity_X': np.float32,
    'angular_velocity_Y': np.float32,
    'angular_velocity_Z': np.float32,
    'linear_acceleration_X': np.float32,
    'linear_acceleration_Y': np.float32,
    'linear_acceleration_Z': np.float32
}

y_cols = {
    'series_id': np.uint32,
    'group_id': np.uint32,
    'surface': str
}

In [6]:
x_trn = pd.read_csv(TRAIN, usecols=x_cols.keys(), dtype=x_cols)
x_tst = pd.read_csv(TEST, usecols=x_cols.keys(), dtype=x_cols)
y_trn = pd.read_csv(TARGET, usecols=y_cols.keys(), dtype=y_cols)

## Features Extraction

Next we prepare a couple of helping utilities to apply feature extraction functions that come with `tsfresh`. We're going to convert sequences of measurements into scalar features and use them to train a classifier.

In [7]:
def part(f, **params):
    """Partially applies the function's keyword parameters."""
    def wrapper(x): return f(x, **params)
    wrapper.__name__ = f.__name__
    return wrapper

In [8]:
class StatsFeatures:
    """Applies list of functions to a single instance of measurements 
    and returns dictionary with computed features.
    """
    def __init__(self, funcs):
        self.funcs = funcs
    
    def __call__(self, data):
        features = {}
        for col in data.columns:
            for func in self.funcs:
                result = func(data[col].values) 
                if hasattr(result, '__len__'):
                    for key, value in result:
                        features[f'{col}__{func.__name__}__{key}'] = value
                else:
                    features[f'{col}__{func.__name__}'] = result
        return features

In [9]:
class SliceFeatures:
    """Takes a slice of values from the original sequence of 
    observations.
    
    There types of slicing are supported:
        * first: take N observations from the beginning of the sequence.
        * middle: take N observations from the middle of the sequence.
        * last: take last N observations from the sequence.
        
    """
    def __init__(self, mode='first', n=5):
        if mode not in {'first', 'middle', 'last'}:
            raise ValueError('unexpected mode')
        self.mode = mode
        self.n = n
        
    def __call__(self, data):
        if self.mode == 'first':
            start, end = 0, self.n
        elif self.mode == 'last':
            start, end = -self.n, len(data)
        elif self.mode == 'middle':
            mid = len(data) // 2
            div, mod = divmod(self.n, 2)
            start, end = mid-div, mid+div+mod
        cols = data.columns
        vec = data.iloc[start:end].values.T.ravel()
        new_cols = [
            f'{col}_{self.mode}{i}' 
            for i in range(self.n) for col in cols]
        return dict(zip(new_cols, vec))

Here is a custom code used to compute features in parallel mode. As it was mentioned previously, you can also use `extract_features` or `extract_relevant_features` functions described in the library's [quick start guide](https://tsfresh.readthedocs.io/en/latest/text/quick_start.html). You only need to override the default settings class with custom implementation. Also, you can use [`calculate_relevance_table`](https://tsfresh.readthedocs.io/en/latest/text/feature_filtering.html) function on top of the dataset with extracted features to keep only relevant onces.

In [10]:
def generate_features(data, features, ignore=None):
    """Extracts tsfresh features from the dataset."""
    
    with Parallel(n_jobs=cpu_count()) as parallel:
        extracted = parallel(delayed(generate_features_for_group)(
            group=group.drop(columns=ignore or []),
            features=features
        ) for _, group in tqdm(data.groupby('series_id')))
    return pd.DataFrame(extracted)

In [11]:
def generate_features_for_group(group, features):
    """Extract tsfresh features from a single measurements group."""
    
    return dict(ChainMap(*[feat(group) for feat in features]))

And here is a list of functions we use to extract the features. The list includes statistical features only. Probably one should add some signal processing here as well to get a better quality.

In [12]:
funcs = (
    mean, median, standard_deviation, variance, 
    skewness, kurtosis, maximum, minimum,
    mean_change, mean_abs_change, count_above_mean, count_below_mean,
    mean_second_derivative_central, sum_of_reoccurring_data_points, 
    abs_energy, sum_values, sample_entropy,
    longest_strike_above_mean, longest_strike_below_mean,
    first_location_of_minimum, first_location_of_maximum,
    *[part(large_standard_deviation, r=r*0.05) for r in range(1, 20)],
    *[part(autocorrelation, lag=lag) for lag in range(1, 25)], 
    *[part(number_peaks, n=n) for n in (1, 2, 3, 5, 7, 10, 25, 50)],
    *[part(c3, lag=lag) for lag in range(1, 5)],
    *[part(quantile, q=q) for q in (.1, .2, .3, .4, .5, .6, .7, .8, .9)],
    part(partial_autocorrelation, param=[
        {'lag': lag} for lag in range(25)
    ]),
    part(agg_autocorrelation, param=[
        {'f_agg': s, 'maxlag': 40} for s in ('mean', 'median', 'var')
    ]),
    part(linear_trend, param=[
        {'attr': a} for a in 
        ('pvalue', 'rvalue', 'intercept', 'slope', 'stderr')
    ])
)

In [13]:
features = [
    StatsFeatures(funcs),
    SliceFeatures('first'),
    SliceFeatures('middle'),
    SliceFeatures('last')
]

In [14]:
ignore = ['series_id', 'measurement_number']

In [15]:
print('Feature extraction on train dataset')
x_trn = generate_features(x_trn, features, ignore=ignore)

Feature extraction on train dataset


In [16]:
print('Feature extraction on test dataset')
x_tst = generate_features(x_tst, features, ignore=ignore)

Feature extraction on test dataset


In [17]:
enc = LabelEncoder()
y_trn = pd.Series(enc.fit_transform(y_trn['surface']))

## The Model

We're going to train a LightGBM classifier using the features shown above. The parameters of the classifier are chosen a bit arbitrarily. One could use a more educated approach and use some kind of parameters search and validation techniques to build a more robust model. 

In [18]:
def accuracy(y_true, y_pred):
    n = len(y_true)
    y_hat = y_pred.reshape(9, n).argmax(axis=0)
    value = (y_true == y_hat).mean()
    return 'accuracy', value, True

In [19]:
model = lgb.LGBMClassifier(n_estimators=3000, learning_rate=0.005,
                           colsample_bytree=0.4, objective='multiclass',
                           num_leaves=500, num_class=9)

In [20]:
x_train, x_valid, y_train, y_valid = train_test_split(x_trn, y_trn, test_size=0.1, random_state=seed)

In [21]:
model.fit(x_train, y_train,
          eval_set=[(x_valid, y_valid)],
          eval_metric=accuracy,
          early_stopping_rounds=300,
          verbose=150)

Training until validation scores don't improve for 300 rounds.
[150]	valid_0's multi_logloss: 1.15655	valid_0's accuracy: 0.813648
[300]	valid_0's multi_logloss: 0.831254	valid_0's accuracy: 0.826772
[450]	valid_0's multi_logloss: 0.656715	valid_0's accuracy: 0.839895
[600]	valid_0's multi_logloss: 0.555689	valid_0's accuracy: 0.847769
[750]	valid_0's multi_logloss: 0.495266	valid_0's accuracy: 0.855643
[900]	valid_0's multi_logloss: 0.457566	valid_0's accuracy: 0.860892
[1050]	valid_0's multi_logloss: 0.433278	valid_0's accuracy: 0.860892
Early stopping, best iteration is:
[823]	valid_0's multi_logloss: 0.474895	valid_0's accuracy: 0.863517


LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=0.4,
        importance_type='split', learning_rate=0.005, max_depth=-1,
        min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
        n_estimators=3000, n_jobs=-1, num_class=9, num_leaves=500,
        objective='multiclass', random_state=None, reg_alpha=0.0,
        reg_lambda=0.0, silent=True, subsample=1.0,
        subsample_for_bin=200000, subsample_freq=0)

In [ ]:
# You could also try to run K-folded validation instead.
#
# k = 5
# test = np.zeros((len(x_tst_rich), 9), dtype=np.float32)
# kfold = StratifiedKFold(n_splits=k, shuffle=True, random_state=seed)
# for i, (trn_idx, val_idx) in enumerate(kfold.split(x_trn_rich.index, y_trn)):
#     x_train = x_trn[x_trn.isin(trn_idx)]
#     x_valid = x_trn[x_trn.isin(val_idx)]
#     y_train = y_trn[y_trn.isin(trn_idx)]
#     y_valid = y_trn[y_trn.isin(val_idx)]
#     model = lgb.LGBMClassifier(n_estimators=3000, learning_rate=0.005,
#                                colsample_bytree=0.4, objective='multiclass',
#                                num_leaves=500, num_class=9)
#     model.fit(x_train, y_train,
#               eval_set=[(x_valid, y_valid)],
#               eval_metric=accuracy,
#               early_stopping_rounds=300,
#               verbose=150)
#     test += model.predict_proba(x_tst_rich)
# test /= k

In [22]:
test = enc.inverse_transform(model.predict(x_tst))

## Submission

In [24]:
submit = pd.read_csv(SAMPLE)
submit['surface'] = test
submit.to_csv('submit.csv', index=None)